In [1]:
import os
import json
!pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import numpy as np

In [2]:
cid ='d8762235995b4ddcbc0df90739df9a63' # Client ID; copy this from your app created on beta.developer.spotify.com
secret = '887a4eccdbde4195bfc0f7f4c472fe70' # Client Secret; copy this from your app
username = '12155311349' # Your Spotify username

#for avaliable scopes see https://developer.spotify.com/web-api/using-scopes/
scope = 'user-library-read playlist-modify-public playlist-read-private'

redirect_uri='https://localhost:8080/callback/'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username,scope,client_id=cid,client_secret=secret,redirect_uri="https://localhost:8080/callback/")

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)


In [3]:
#sourcePlaylistID = 'https://open.spotify.com/playlist/5PlFP6HbcJKMqAZ557UgSG?si=8b8c10a0e8d748c4' #My Playlist
sourcePlaylistID ='https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=b4fc7abf24d943b7' #Rock Playlist
sourcePlaylist = sp.user_playlist(username, sourcePlaylistID);
tracks = sourcePlaylist["tracks"];
songs = tracks["items"];

#Putting Track ID, Track Name, Album, and artist name in matrix's
track_ids = [] 
track_names = []
album = []
artist =[]

for i in range(0, len(songs)):
    if songs[i]['track']['id'] != None: # Removes the local tracks in your playlist if there is any
        track_ids.append(songs[i]['track']['id'])
        track_names.append(songs[i]['track']['name'])
        album.append(songs[i]['track']['album']['name'])
        artist.append(songs[i]['track']['artists'][0]['name'])

features = []
for i in range(0,len(songs)):
    audio_features = sp.audio_features(track_ids[i])
    for track in audio_features:
      
      if track is None:
        print(track)
        features.append({'danceability': 0, 'energy': 0, 'key': 0, 'loudness': 0, 'mode': 0, 'speechiness': 0, 'acousticness': 0, 'instrumentalness': 0, 'liveness': 0, 'valence': 0, 'tempo': 0, 'type': 'audio_features', 'id': '00000', 'uri': 'spotify:track:0', 'track_href': 'https://api.spotify.com/', 'analysis_url': 'https://api.spotify.com/', 'duration_ms': 0, 'time_signature': 0})
      else:
        features.append(track)
playlist_df = pd.DataFrame(features, index = track_names)
all_df = playlist_df[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','id']]
#Useful for Displaying the Data
playlist_matrix = playlist_df[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]

In [4]:
playlist_matrix.head()
#playlist_matrix for the whole table

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
Over the Hills and Far Away - Remaster,0.402,0.691,7,-10.274,1,0.0691,0.06790,0.133000,0.0955,0.206,96.950
Seven Nation Army,0.743,0.446,4,-7.807,0,0.0683,0.00390,0.118000,0.3370,0.278,123.596
The Chain - 2004 Remaster,0.545,0.670,9,-8.810,1,0.0496,0.00900,0.000822,0.0451,0.481,151.553
Hotel California - 2013 Remaster,0.579,0.508,2,-9.484,1,0.0270,0.00574,0.000494,0.0575,0.609,147.125
Another One Bites The Dust - Remastered 2011,0.933,0.528,5,-6.472,0,0.1620,0.11200,0.329000,0.1630,0.756,109.975


In [5]:
# playlist_danceability = playlist_df['danceability']
# print(playlist_danceability)

#Getting all the values into a matrix
danceability_float = playlist_matrix['danceability'].values
# print(danceability_float)

# danceability_mean = danceability_float.mean()

In [6]:
#Checking the Data
playlist_matrix.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.540930,0.740600,5.340000,-7.846270,0.770000,0.062402,0.129916,0.056157,0.176222,0.591060,124.833830
std,0.138606,0.169455,3.414822,2.810143,0.422953,0.056547,0.164956,0.134900,0.118285,0.216911,22.988906
min,0.249000,0.366000,0.000000,-18.064000,0.000000,0.024900,0.000018,0.000000,0.024400,0.104000,78.031000
25%,0.451250,0.630000,2.000000,-9.377500,1.000000,0.033300,0.003802,0.000031,0.086900,0.438750,112.731000
50%,0.539000,0.748000,5.000000,-7.567500,1.000000,0.044600,0.061050,0.000734,0.138500,0.606000,124.374000
75%,0.643500,0.901250,9.000000,-5.690750,1.000000,0.069525,0.202250,0.020800,0.268250,0.765000,137.458500
max,0.933000,0.987000,11.000000,-2.788000,1.000000,0.405000,0.619000,0.836000,0.659000,0.964000,195.815000


In [7]:
#Hot Encoding our Catagories (converts data to numerical represnetation)
playlist_matrix = pd.get_dummies(playlist_matrix)
playlist_matrix.iloc[:,:].head(5)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
Over the Hills and Far Away - Remaster,0.402,0.691,7,-10.274,1,0.0691,0.06790,0.133000,0.0955,0.206,96.950
Seven Nation Army,0.743,0.446,4,-7.807,0,0.0683,0.00390,0.118000,0.3370,0.278,123.596
The Chain - 2004 Remaster,0.545,0.670,9,-8.810,1,0.0496,0.00900,0.000822,0.0451,0.481,151.553
Hotel California - 2013 Remaster,0.579,0.508,2,-9.484,1,0.0270,0.00574,0.000494,0.0575,0.609,147.125
Another One Bites The Dust - Remastered 2011,0.933,0.528,5,-6.472,0,0.1620,0.11200,0.329000,0.1630,0.756,109.975


In [8]:
# # Labels are the values we want to predict
# labels = np.array(playlist_matrix['Genres'])
# # Remove the labels from the features
# # axis 1 refers to the columns
# playlist_matrix= playlist_matrix.drop('actual', axis = 1)
# # Saving feature names for later use
# playlist_list = list(playlist_matrix.columns)
# Convert to numpy array
playlist_matrix.info()
# playlist_matrix = np.array(playlist_matrix)


<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, Over the Hills and Far Away - Remaster to Smoke on the Water
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      100 non-null    float64
 1   energy            100 non-null    float64
 2   key               100 non-null    int64  
 3   loudness          100 non-null    float64
 4   mode              100 non-null    int64  
 5   speechiness       100 non-null    float64
 6   acousticness      100 non-null    float64
 7   instrumentalness  100 non-null    float64
 8   liveness          100 non-null    float64
 9   valence           100 non-null    float64
 10  tempo             100 non-null    float64
dtypes: float64(9), int64(2)
memory usage: 9.4+ KB


In [9]:
import plotly.graph_objects as go
import plotly.offline as pyo

plot_rock = playlist_df[['danceability','energy','mode','speechiness','acousticness','instrumentalness','liveness','valence']]

categories = ['danceability','energy','mode','speechiness','acousticness','instrumentalness','liveness','valence']
categories = [*categories, categories[0]]

mean_data = plot_rock.mean().values
# restaurant_2 = [5, 5, 4, 5, 2]
# restaurant_3 = [3, 4, 5, 3, 5]
mean_data = [*mean_data, mean_data[0]]
# restaurant_2 = [*restaurant_2, restaurant_2[0]]
# restaurant_3 = [*restaurant_3, restaurant_3[0]]


fig = go.Figure(
    data=[
        go.Scatterpolar(r=mean_data, theta=categories, name='Rock Music'),
#         go.Scatterpolar(r=restaurant_2, theta=categories, fill='toself', name='Restaurant 2'),
#         go.Scatterpolar(r=restaurant_3, theta=categories, fill='toself', name='Restaurant 3')
    ],
    layout=go.Layout(
        title=go.layout.Title(text='Restaurant comparison'),
        polar={'radialaxis': {'visible': True}},
        showlegend=True,
    )
)

pyo.plot(fig)

'temp-plot.html'

In [10]:
from math import pi

# number of variable
categories=list(playlist_matrix)[1:]
N = len(categories)
N

10

In [11]:
playlist_matrix.mean().values

array([ 5.40930000e-01,  7.40600000e-01,  5.34000000e+00, -7.84627000e+00,
        7.70000000e-01,  6.24020000e-02,  1.29916058e-01,  5.61573407e-02,
        1.76222000e-01,  5.91060000e-01,  1.24833830e+02])